In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd /content/gdrive/MyDrive/SemEval2023/SemEval2022-Task10/re_run

/content/gdrive/.shortcut-targets-by-id/1lC-ZKLaCDQyfLcof2Ak7FDa6IvTt318A/SemEval2023/SemEval2022-Task10/re_run


In [4]:
import pandas as pd

data = pd.read_csv("../Data/starting_ki/train_all_tasks.csv")
# dfa = pd.read_csv("Data/dev_task_a_entries.csv")
# dfb = pd.read_csv("Data/dev_task_b_entries.csv")
# dfc = pd.read_csv("Data/dev_task_c_entries.csv")

In [5]:
import json

# with open('../Results/TaskB/lexicon_pmi.json') as fin:
#     lexicon_pmi = json.load(fin)

# with open('../Results/TaskB/lexicon_shapley.json') as fin:
#     lexicon_shapley = json.load(fin)

# with open('../Results/TaskB/lexicon_shapley_augmented_v2_shap.json') as fin:
#     lexicon_shapley_augmented_v2_shap = json.load(fin)

# with open('../Results/TaskB/lexicon_shapley_augmented_v3_gptj.json') as fin:
#     lexicon_shapley_augmented_v3_gptj = json.load(fin)

# with open('../Results/TaskB/lexicon_shapley_augmented_v4_tweetbert.json') as fin:
#     lexicon_shapley_augmented_v4_tweetbert = json.load(fin)

In [6]:
data

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,"""GRAPHIC Germany - Muslim ""refugee"" stabbing h...",not sexist,none,none
13997,sexism2022_english-3320,Lol I imagine there would be simps that are li...,not sexist,none,none
13998,sexism2022_english-13309,"It's not, the girls I go on dates with don't k...",not sexist,none,none


In [7]:
task = "1c"

In [8]:
if not task.endswith("a"):
  data = data[data["label_category"]!="none"]

In [9]:
if task.endswith("a"):
  label_column = "label_sexist"
elif task.endswith("b"):
  label_column = "label_category"
elif task.endswith("c"):
  label_column = "label_vector"


label_column

'label_vector'

In [10]:
len(data)

3398

In [11]:
data[["text", label_column]].groupby(label_column).count()

,text
label_vector,
1.1 threats of harm,56
1.2 incitement and encouragement of harm,254
2.1 descriptive attacks,717
2.2 aggressive and emotive attacks,673
2.3 dehumanising attacks & overt sexual objectification,200
"3.1 casual use of gendered slurs, profanities, and insults",637
3.2 immutable gender differences and gender stereotypes,417
3.3 backhanded gendered compliments,64
3.4 condescending explanations or unwelcome advice,47


In [12]:
# label_values = data[label_column].unique()
# label_values
if label_column == "label_sexist":
  label_values = [
      'sexist',
      'not sexist',
  ]

  label_map = {
      0: 'not sexist',
      1: 'sexist', 
      'not sexist':0,
      'sexist':1,
  }
elif label_column == "label_category":
  label_values = [
      '1. threats, plans to harm and incitement',
      '2. derogation',
      '3. animosity',
      '4. prejudiced discussions',
  ]

  label_map = {
      0: '1. threats, plans to harm and incitement',
      1: '2. derogation', 
      2: '3. animosity', 
      3: '4. prejudiced discussions',
      '1. threats, plans to harm and incitement':0,
      '2. derogation':1,
      '3. animosity':2,
      '4. prejudiced discussions':3,
  }
elif label_column == "label_vector":
  label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

  label_map = {
      0: '1.1 threats of harm',
      1: '1.2 incitement and encouragement of harm',
      2: '2.1 descriptive attacks',
      3: '2.2 aggressive and emotive attacks',
      4: '2.3 dehumanising attacks & overt sexual objectification',
      5: '3.1 casual use of gendered slurs, profanities, and insults',
      6: '3.2 immutable gender differences and gender stereotypes',
      7: '3.3 backhanded gendered compliments',
      8: '3.4 condescending explanations or unwelcome advice',
      9: '4.1 supporting mistreatment of individual women',
      10: '4.2 supporting systemic discrimination against women as a group',
      '1.1 threats of harm': 0,
      '1.2 incitement and encouragement of harm': 1,
      '2.1 descriptive attacks': 2,
      '2.2 aggressive and emotive attacks': 3,
      '2.3 dehumanising attacks & overt sexual objectification': 4,
      '3.1 casual use of gendered slurs, profanities, and insults': 5,
      '3.2 immutable gender differences and gender stereotypes': 6,
      '3.3 backhanded gendered compliments': 7,
      '3.4 condescending explanations or unwelcome advice': 8,
      '4.1 supporting mistreatment of individual women': 9,
      '4.2 supporting systemic discrimination against women as a group': 10,
  }
else:
  raise Exception("Unknown label column")

# Prepare Data

In [13]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.9 MB/s eta 0:00:00


In [14]:
import transformers

In [15]:
from transformers import BertTokenizer
from transformers import AutoModel, AutoTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

# print('Original Text: ', texts[0], '\n')
# print('Tokenized Text: ', tokenizer.tokenize(texts[0]), '\n')
# print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))


In [16]:
# !pip3 install -q nltk emoji==0.6.0

In [17]:
# from emoji import demojize
# from nltk.tokenize import TweetTokenizer


# nltktokenizer = TweetTokenizer()


# def normalizeToken(token):
#     lowercased_token = token.lower()
#     if token.startswith("@"):
#         return "@USER"
#     elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
#         return "HTTPURL"
#     elif len(token) == 1:
#         return demojize(token)
#     else:
#         if token == "’":
#             return "'"
#         elif token == "…":
#             return "..."
#         else:
#             return token


# def normalizeTweet(tweet):
#     tokens = nltktokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
#     normTweet = " ".join([normalizeToken(token) for token in tokens])

#     normTweet = (
#         normTweet.replace("cannot ", "can not ")
#         .replace("n't ", " n't ")
#         .replace("n 't ", " n't ")
#         .replace("ca n't", "can't")
#         .replace("ai n't", "ain't")
#     )
#     normTweet = (
#         normTweet.replace("'m ", " 'm ")
#         .replace("'re ", " 're ")
#         .replace("'s ", " 's ")
#         .replace("'ll ", " 'll ")
#         .replace("'d ", " 'd ")
#         .replace("'ve ", " 've ")
#     )
#     normTweet = (
#         normTweet.replace(" p . m .", "  p.m.")
#         .replace(" p . m ", " p.m ")
#         .replace(" a . m .", " a.m.")
#         .replace(" a . m ", " a.m ")
#     )

#     return " ".join(normTweet.split())

In [18]:
# normalizeTweet("DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier 😢")


In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
from nltk.tokenize import  word_tokenize

In [21]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from nltk.tokenize import TweetTokenizer
nltktokenizer = TweetTokenizer()

def word_tokenize(sent):
  sent = sent.lower()
  words = nltktokenizer.tokenize(sent)
  return words

def label_to_number(label_map, label):
  return label_map[label]

def get_tensor(df, lexicons):
  texts = df['text'].values
  labels = [label_to_number(label_map, d) for d in df[label_column].values]

  text_ids = [tokenizer.encode(text, max_length=100, padding='max_length', truncation=True) for text in texts]
  text_ids_lengths = [len(text_ids[i]) for i in range(len(text_ids))]

  att_masks = []
  for ids in text_ids:
      masks = [int(id > 0) for id in ids]
      att_masks.append(masks)

  lexcon_emb = []
  for text in texts:
    words = word_tokenize(text)

    embs = []
    for w in words:
      e = []
      for label in lexicons:
        if w in lexicons[label]:
          e.append(1.0)
        else:
          e.append(0.0)
      embs.append(e)

    while len(embs) < 100:
      embs.append([0 for l in lexicons])
    lexcon_emb.append(embs)

  tensor_x = torch.tensor(text_ids)
  tensor_y = torch.tensor(labels)
  tensor_m = torch.tensor(att_masks)
  tensor_l = torch.tensor(lexcon_emb)

  return tensor_x, tensor_y, tensor_m, tensor_l


def get_loader(train, test, val, lexicons, batch_size=32):
  
  train_x, train_y, train_m, train_l = get_tensor(train, lexicons)
  test_x, test_y, test_m, test_l = get_tensor(test, lexicons)
  val_x, val_y, val_m, val_l = get_tensor(train, lexicons)

  train_data = TensorDataset(train_x, train_m, train_y, train_l)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  val_data = TensorDataset(val_x, val_m, val_y, val_l)
  val_sampler = SequentialSampler(val_data)
  val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

  test_data = TensorDataset(test_x, test_m, test_y, test_l)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  return train_dataloader, test_dataloader, val_dataloader

## Load Model

In [22]:
from transformers import BertForSequenceClassification
from transformers import BertPreTrainedModel, BertModel
from typing import List, Optional, Tuple, Union

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutput


class CustomBertForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(in_features=config.hidden_size+config.num_labels, out_features=config.num_labels)
        self.lstm = nn.LSTM(input_size=config.num_labels, hidden_size=config.num_labels, num_layers=1, batch_first=True)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        lexicon_emb: Optional[torch.Tensor] = None
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:

        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)

        lstm_output, (ht, ct) = self.lstm(lexicon_emb)
        ht = torch.squeeze(ht, dim=0)
        
        concated_ouput = torch.cat((pooled_output, ht), 1)
        
        logits = self.classifier(concated_ouput)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [23]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import AutoModel, AutoTokenizer

# num_labels = len(set(labels))

# model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels, output_attentions=False, output_hidden_states=False)
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels, output_attentions=False, output_hidden_states=False)
# model = BertForSequenceClassification.from_pretrained("vinai/bertweet-large", num_labels=num_labels, output_attentions=False, output_hidden_states=False)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

# print(device)

In [25]:
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print('Number of trainable parameters:', count_parameters(model), '\n', model)


## Fine-tune

In [26]:
from transformers import get_linear_schedule_with_warmup
import time
import numpy as np
import random
from tqdm.notebook import tqdm
import os


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def run(train_dataloader, val_dataloader, model, optimizer, scheduler, num_epochs=5, add_lexicon=True, tmp_out_dir="./Models/_tmp"):
  train_losses = []
  val_losses = []
  num_mb_train = len(train_dataloader)
  num_mb_val = len(val_dataloader)

  if num_mb_val == 0:
      num_mb_val = 1

  best_loss = None

  for n in range(num_epochs):
      train_loss = 0
      val_loss = 0
      start_time = time.time()
      
      for k, (mb_x, mb_m, mb_y, mb_l) in enumerate(train_dataloader):
          optimizer.zero_grad()
          model.train()
          
          mb_x = mb_x.to(device)
          mb_m = mb_m.to(device)
          mb_y = mb_y.to(device)
          mb_l = mb_l.to(device)
          
          if add_lexicon:
            outputs = model(mb_x, attention_mask=mb_m, labels=mb_y, lexicon_emb=mb_l)
          else:
            outputs = model(mb_x, attention_mask=mb_m, labels=mb_y)
          
          loss = outputs[0]
          #loss = model_loss(outputs[1], mb_y)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()
          
          train_loss += loss.data / num_mb_train
      
      print ("\nTrain loss after itaration %i: %f" % (n+1, train_loss))
      train_losses.append(train_loss.cpu())
      
      with torch.no_grad():
          model.eval()
          
          for k, (mb_x, mb_m, mb_y, mb_l) in enumerate(val_dataloader):
              mb_x = mb_x.to(device)
              mb_m = mb_m.to(device)
              mb_y = mb_y.to(device)
              mb_l = mb_l.to(device)
          
              if add_lexicon:
                outputs = model(mb_x, attention_mask=mb_m, labels=mb_y, lexicon_emb=mb_l)
              else:
                outputs = model(mb_x, attention_mask=mb_m, labels=mb_y)
              
              loss = outputs[0]
              #loss = model_loss(outputs[1], mb_y)
              
              val_loss += loss.data / num_mb_val

          if best_loss is None or best_loss >= val_loss:
              
              if not os.path.exists(tmp_out_dir):
                  os.makedirs(tmp_out_dir)
                  
              model.save_pretrained(tmp_out_dir)
              # tokenizer.save_pretrained(tmp_out_dir)

              best_loss = val_loss

          print("Validation loss after itaration %i: %f" % (n+1, val_loss))
          val_losses.append(val_loss.cpu())
      
      end_time = time.time()
      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      print(f'Time: {epoch_mins}m {epoch_secs}s')

  if add_lexicon:
    model = CustomBertForSequenceClassification.from_pretrained(tmp_out_dir)
  else:
    model = BertForSequenceClassification.from_pretrained(tmp_out_dir)

  model = model.to(device)
  return model, train_losses, val_losses

def fine_tune(train_dataloader, val_dataloader, seed_val = 42, num_epochs=5, add_lexicon=True, tmp_out_dir="./Models/_tmp"):
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  num_labels = len(label_values)
  if add_lexicon:
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels, output_attentions=False, output_hidden_states=False)
  else:
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels, output_attentions=False, output_hidden_states=False)

  model = model.to(device)

  learning_rate = 1e-5
  adam_epsilon = 1e-8

  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.2},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
  ]

  optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

  total_steps = len(train_dataloader) * num_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
  return run(train_dataloader, val_dataloader, model, optimizer, scheduler, num_epochs, add_lexicon, tmp_out_dir)



In [27]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

def get_prediction(model, test_dataloader, add_lexicon = True):
  outputs = []
  test_y = []
  with torch.no_grad():
      model.eval()
      for k, (mb_x, mb_m, mb_y, mb_l) in enumerate(test_dataloader):
          mb_x = mb_x.to(device)
          mb_m = mb_m.to(device)
          mb_l = mb_l.to(device)
          if add_lexicon:
            output = model(mb_x, attention_mask=mb_m, lexicon_emb=mb_l)
          else:
            output = model(mb_x, attention_mask=mb_m)
          outputs.append(output[0].to('cpu'))
          test_y.append(mb_y)

  outputs = torch.cat(outputs)


  _, predicted_values = torch.max(outputs, 1)
  predicted_values = predicted_values.numpy()
  true_values = torch.concat(test_y).numpy()

  test_accuracy = np.sum(predicted_values == true_values) / len(true_values)

  # print(classification_report(true_values, predicted_values, target_names=[str(l) for l in label_values]))
  p, r, f1, sup = precision_recall_fscore_support(true_values, predicted_values, average="macro")

  return f1, p, r

In [28]:
# !pip install numba

In [29]:
from numba import cuda

def train_with_lexicons(i, lex_path=None, num_epochs=5, add_lexicon=True):
  
  # d = cuda.get_current_device() 
  # d.reset()

  train = pd.read_csv(f"Data/{i}_train.csv")
  test = pd.read_csv(f"Data/{i}_test.csv")
  val = pd.read_csv(f"Data/{i}_val.csv")

  if lex_path is None:
    lexicons = {}
    for l in label_values:
      lexicons[l] = {}
  else:
    with open(f'{lex_path}_{i}.json') as fin:
      lexicons = json.load(fin)

  train_dataloader, test_dataloader, val_dataloader = get_loader(train,test, val, lexicons, batch_size=32)
  model, train_losses, val_losses = fine_tune(train_dataloader, val_dataloader, num_epochs=num_epochs, add_lexicon=add_lexicon, tmp_out_dir="/content/Models/_tmp")
  f1, p, r = get_prediction(model, test_dataloader, add_lexicon=add_lexicon)
  print(f1, p, r)
  # all_f1.append(f1)
  # all_p.append(p)
  # all_r.append(r)
  
  # print(f"F1:{np.mean(all_f1):.3f}±{np.std(all_f1):.3f}")
  # print(f"P:{np.mean(all_p):.3f}±{np.std(all_p):.3f}, R: {np.mean(all_r):.3f}±{np.std(all_r):.3f}")

# train_with_lexicons("Results/TaskB/lexicon_pmi_train")

In [30]:
len(label_values)

11

In [31]:
# train_with_lexicons(0, None, num_epochs=10, add_lexicon=False)

In [32]:
# train_with_lexicons(1, None, num_epochs=10, add_lexicon=False)

In [33]:
# train_with_lexicons(2, None, num_epochs=10, add_lexicon=False)

In [34]:
# train_with_lexicons(3, None, num_epochs=10, add_lexicon=False)

In [35]:
# train_with_lexicons(4, None, num_epochs=10, add_lexicon=False)

In [ ]:
# train_with_lexicons("Results/TaskC/lexicon_shapley_train", num_epochs=5, add_lexicon=True)

In [ ]:
train_with_lexicons(0, "Results/TaskC/lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.113035
Validation loss after itaration 1: 1.927315
Time: 0m 56s

Train loss after itaration 2: 1.877714
Validation loss after itaration 2: 1.739455
Time: 0m 58s

Train loss after itaration 3: 1.716143
Validation loss after itaration 3: 1.560806
Time: 1m 0s

Train loss after itaration 4: 1.567508
Validation loss after itaration 4: 1.420192
Time: 1m 1s

Train loss after itaration 5: 1.458876
Validation loss after itaration 5: 1.301878
Time: 1m 2s

Train loss after itaration 6: 1.354594
Validation loss after itaration 6: 1.214576
Time: 1m 1s

Train loss after itaration 7: 1.274687
Validation loss after itaration 7: 1.136562
Time: 1m 1s

Train loss after itaration 8: 1.209425
Validation loss after itaration 8: 1.070438
Time: 1m 2s

Train loss after itaration 9: 1.167812
Validation loss after itaration 9: 1.041291
Time: 1m 1s

Train loss after itaration 10: 1.137006
Validation loss after itaration 10: 1.026929
Time: 1m 1s
0.28632686494585946 0.34872676828122

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
train_with_lexicons(1, "Results/TaskC/lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.107555
Validation loss after itaration 1: 1.925259
Time: 1m 2s

Train loss after itaration 2: 1.874163
Validation loss after itaration 2: 1.724045
Time: 1m 1s

Train loss after itaration 3: 1.715193
Validation loss after itaration 3: 1.579504
Time: 1m 1s

Train loss after itaration 4: 1.592649
Validation loss after itaration 4: 1.456519
Time: 1m 2s

Train loss after itaration 5: 1.489746
Validation loss after itaration 5: 1.351977
Time: 1m 1s

Train loss after itaration 6: 1.403869
Validation loss after itaration 6: 1.256721
Time: 1m 1s

Train loss after itaration 7: 1.314182
Validation loss after itaration 7: 1.192066
Time: 1m 2s

Train loss after itaration 8: 1.252471
Validation loss after itaration 8: 1.125986
Time: 1m 1s

Train loss after itaration 9: 1.217288
Validation loss after itaration 9: 1.095864
Time: 1m 1s

Train loss after itaration 10: 1.187057
Validation loss after itaration 10: 1.079417
Time: 1m 2s
0.2754581357138901 0.2726235593036556 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
train_with_lexicons(2, "Results/TaskC/lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.114479
Validation loss after itaration 1: 1.916798
Time: 1m 2s

Train loss after itaration 2: 1.864374
Validation loss after itaration 2: 1.730587
Time: 1m 1s

Train loss after itaration 3: 1.705163
Validation loss after itaration 3: 1.565059
Time: 1m 1s

Train loss after itaration 4: 1.576140
Validation loss after itaration 4: 1.445557
Time: 1m 1s

Train loss after itaration 5: 1.472174
Validation loss after itaration 5: 1.331511
Time: 1m 2s

Train loss after itaration 6: 1.378659
Validation loss after itaration 6: 1.230747
Time: 1m 1s

Train loss after itaration 7: 1.283489
Validation loss after itaration 7: 1.151307
Time: 1m 1s

Train loss after itaration 8: 1.222191
Validation loss after itaration 8: 1.096450
Time: 1m 1s

Train loss after itaration 9: 1.174708
Validation loss after itaration 9: 1.058758
Time: 1m 1s

Train loss after itaration 10: 1.140319
Validation loss after itaration 10: 1.047187
Time: 1m 1s
0.236895428274869 0.24405792100100543 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
train_with_lexicons(3, "Results/TaskC/lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.113018
Validation loss after itaration 1: 1.916057
Time: 1m 2s

Train loss after itaration 2: 1.868017
Validation loss after itaration 2: 1.720219
Time: 1m 1s

Train loss after itaration 3: 1.722841
Validation loss after itaration 3: 1.581476
Time: 1m 1s

Train loss after itaration 4: 1.593079
Validation loss after itaration 4: 1.453092
Time: 1m 1s

Train loss after itaration 5: 1.491830
Validation loss after itaration 5: 1.343762
Time: 1m 1s

Train loss after itaration 6: 1.385381
Validation loss after itaration 6: 1.248413
Time: 1m 1s

Train loss after itaration 7: 1.312262
Validation loss after itaration 7: 1.167240
Time: 1m 1s

Train loss after itaration 8: 1.235924
Validation loss after itaration 8: 1.115081
Time: 1m 1s

Train loss after itaration 9: 1.195722
Validation loss after itaration 9: 1.080804
Time: 1m 1s

Train loss after itaration 10: 1.168869
Validation loss after itaration 10: 1.068549
Time: 1m 1s
0.2489954383044023 0.25745932066277394

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
train_with_lexicons(4, "Results/TaskC/lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.104811
Validation loss after itaration 1: 1.909382
Time: 0m 58s

Train loss after itaration 2: 1.865254
Validation loss after itaration 2: 1.725273
Time: 0m 59s

Train loss after itaration 3: 1.709884
Validation loss after itaration 3: 1.573564
Time: 0m 59s

Train loss after itaration 4: 1.580468
Validation loss after itaration 4: 1.436032
Time: 0m 58s

Train loss after itaration 5: 1.466594
Validation loss after itaration 5: 1.308509
Time: 0m 59s

Train loss after itaration 6: 1.355721
Validation loss after itaration 6: 1.200842
Time: 0m 58s

Train loss after itaration 7: 1.267681
Validation loss after itaration 7: 1.122772
Time: 0m 59s

Train loss after itaration 8: 1.208785
Validation loss after itaration 8: 1.067375
Time: 0m 59s

Train loss after itaration 9: 1.156789
Validation loss after itaration 9: 1.030578
Time: 0m 59s

Train loss after itaration 10: 1.122581
Validation loss after itaration 10: 1.020422
Time: 0m 58s
0.26953512400084595 0.273247

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# train_with_lexicons("Results/TaskC/augmented_lexicon_shapley_train", num_epochs=5, add_lexicon=True)

In [ ]:
train_with_lexicons(0, "Results/TaskC/augmented_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.113113
Validation loss after itaration 1: 1.927775
Time: 0m 59s

Train loss after itaration 2: 1.875967
Validation loss after itaration 2: 1.739106
Time: 0m 59s

Train loss after itaration 3: 1.712771
Validation loss after itaration 3: 1.558586
Time: 0m 59s

Train loss after itaration 4: 1.563253
Validation loss after itaration 4: 1.415928
Time: 0m 59s

Train loss after itaration 5: 1.454532
Validation loss after itaration 5: 1.296656
Time: 0m 59s

Train loss after itaration 6: 1.350836
Validation loss after itaration 6: 1.214232
Time: 0m 59s

Train loss after itaration 7: 1.271296
Validation loss after itaration 7: 1.134653
Time: 0m 59s

Train loss after itaration 8: 1.205214
Validation loss after itaration 8: 1.067697
Time: 0m 59s

Train loss after itaration 9: 1.163044
Validation loss after itaration 9: 1.038234
Time: 0m 59s

Train loss after itaration 10: 1.133758
Validation loss after itaration 10: 1.024014
Time: 0m 59s
0.2866668611770293 0.3490737

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
train_with_lexicons(1, "Results/TaskC/augmented_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.107555
Validation loss after itaration 1: 1.925259
Time: 1m 2s

Train loss after itaration 2: 1.874283
Validation loss after itaration 2: 1.725154
Time: 1m 3s

Train loss after itaration 3: 1.715503
Validation loss after itaration 3: 1.580983
Time: 1m 4s

Train loss after itaration 4: 1.592632
Validation loss after itaration 4: 1.456498
Time: 1m 3s

Train loss after itaration 5: 1.490418
Validation loss after itaration 5: 1.354766
Time: 1m 4s

Train loss after itaration 6: 1.407711
Validation loss after itaration 6: 1.261166
Time: 1m 3s

Train loss after itaration 7: 1.319552
Validation loss after itaration 7: 1.195662
Time: 1m 3s

Train loss after itaration 8: 1.258192
Validation loss after itaration 8: 1.130137
Time: 1m 3s

Train loss after itaration 9: 1.221866
Validation loss after itaration 9: 1.101694
Time: 1m 3s

Train loss after itaration 10: 1.191557
Validation loss after itaration 10: 1.084750
Time: 1m 3s
0.26816102756608595 0.2663593500287049

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
train_with_lexicons(2, "Results/TaskC/augmented_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.114472
Validation loss after itaration 1: 1.916713
Time: 1m 4s

Train loss after itaration 2: 1.863006
Validation loss after itaration 2: 1.734579
Time: 1m 3s

Train loss after itaration 3: 1.706474
Validation loss after itaration 3: 1.567371
Time: 1m 3s

Train loss after itaration 4: 1.576934
Validation loss after itaration 4: 1.448379
Time: 1m 3s

Train loss after itaration 5: 1.471991
Validation loss after itaration 5: 1.329091
Time: 1m 3s

Train loss after itaration 6: 1.377586
Validation loss after itaration 6: 1.229761
Time: 1m 4s

Train loss after itaration 7: 1.283394
Validation loss after itaration 7: 1.150598
Time: 1m 3s

Train loss after itaration 8: 1.221184
Validation loss after itaration 8: 1.095513
Time: 1m 3s

Train loss after itaration 9: 1.172466
Validation loss after itaration 9: 1.056936
Time: 1m 3s

Train loss after itaration 10: 1.138940
Validation loss after itaration 10: 1.045058
Time: 1m 4s
0.2408551450847834 0.2485332124700008 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
train_with_lexicons(3, "Results/TaskC/augmented_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.113019
Validation loss after itaration 1: 1.916066
Time: 1m 4s

Train loss after itaration 2: 1.868072
Validation loss after itaration 2: 1.720920
Time: 1m 3s

Train loss after itaration 3: 1.722684
Validation loss after itaration 3: 1.583214
Time: 1m 3s

Train loss after itaration 4: 1.593446
Validation loss after itaration 4: 1.452900
Time: 1m 3s

Train loss after itaration 5: 1.491029
Validation loss after itaration 5: 1.340801
Time: 1m 4s

Train loss after itaration 6: 1.382604
Validation loss after itaration 6: 1.246422
Time: 1m 3s

Train loss after itaration 7: 1.309726
Validation loss after itaration 7: 1.166614
Time: 1m 3s

Train loss after itaration 8: 1.233793
Validation loss after itaration 8: 1.113083
Time: 1m 3s

Train loss after itaration 9: 1.192590
Validation loss after itaration 9: 1.078386
Time: 1m 3s

Train loss after itaration 10: 1.165869
Validation loss after itaration 10: 1.066768
Time: 1m 3s
0.24701543667241754 0.2535104800504221

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
train_with_lexicons(4, "Results/TaskC/augmented_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from


Train loss after itaration 1: 2.104811
Validation loss after itaration 1: 1.909382
Time: 1m 4s

Train loss after itaration 2: 1.865251
Validation loss after itaration 2: 1.725304
Time: 1m 3s

Train loss after itaration 3: 1.709854
Validation loss after itaration 3: 1.573952
Time: 1m 3s

Train loss after itaration 4: 1.580456
Validation loss after itaration 4: 1.436925
Time: 1m 3s

Train loss after itaration 5: 1.466616
Validation loss after itaration 5: 1.308494
Time: 1m 4s

Train loss after itaration 6: 1.355834
Validation loss after itaration 6: 1.200461
Time: 1m 3s

Train loss after itaration 7: 1.267822
Validation loss after itaration 7: 1.123478
Time: 1m 3s

Train loss after itaration 8: 1.209195
Validation loss after itaration 8: 1.067558
Time: 1m 3s

Train loss after itaration 9: 1.157644
Validation loss after itaration 9: 1.031111
Time: 1m 3s

Train loss after itaration 10: 1.123063
Validation loss after itaration 10: 1.021130
Time: 1m 3s
0.2661079937123043 0.2679131825749231 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# train_with_lexicons(0, "Results/TaskC/gptj_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(1, "Results/TaskC/gptj_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(2, "Results/TaskC/gptj_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(3, "Results/TaskC/gptj_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(4, "Results/TaskC/gptj_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(0, "Results/TaskC/bertweet_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(1, "Results/TaskC/bertweet_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(2, "Results/TaskC/bertweet_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(3, "Results/TaskC/bertweet_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

In [ ]:
# train_with_lexicons(4, "Results/TaskC/bertweet_lexicon_shapley_train", num_epochs=10, add_lexicon=True)

## Create Submission

In [ ]:
# ls ../Data/starting_ki/train_all_tasks.csv

In [ ]:
# dftrain = pd.read_csv("../Data/starting_ki/train_all_tasks.csv")
# dfdev = pd.read_csv("../Data/dev_task_b_entries.csv")
# dftest = pd.read_csv("../Data/test_task_b_entries.csv")

In [ ]:
df = dftrain
texts = df['text'].values

In [ ]:
text_ids = [tokenizer.encode(text, max_length=100, padding='max_length') for text in texts]

In [ ]:
att_masks = []
for ids in text_ids:
    masks = [int(id > 0) for id in ids]
    att_masks.append(masks)

In [ ]:
lexcon_emb = []
for text in texts:
  words = word_tokenize(text)

  embs = []
  for w in words:
    e = []
    for label in lexicons:
      if w in lexicons[label]:
        e.append(1.0)
      else:
        e.append(0.0)
    embs.append(e)

  while len(embs) < 100:
    embs.append([0 for l in lexicons])
  lexcon_emb.append(embs)


In [ ]:
test_x = torch.tensor(text_ids);
test_m = torch.tensor(att_masks);
test_l = torch.tensor(lexcon_emb);

In [ ]:
len(df), test_x.shape

In [ ]:
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32
test_data = TensorDataset(test_x, test_m, test_l)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

outputs = []
with torch.no_grad():
    model.eval()
    for k, (mb_x, mb_m, mb_l) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        mb_x = mb_x.to(device)
        mb_m = mb_m.to(device)
        mb_l = mb_l.to(device)
        output = model(mb_x, attention_mask=mb_m, lexicon_emb=mb_l)
        outputs.append(output[0].to('cpu'))

outputs = torch.cat(outputs)
_, predicted_values = torch.max(outputs, 1)
predicted_values = predicted_values.numpy()

In [ ]:
def number_to_label(label):
  return label_map[label]

predicted_labels = list(map(number_to_label, predicted_values))

In [ ]:
len(predicted_labels)

In [ ]:
df["label_pred"] = predicted_labels

In [ ]:
df

In [ ]:
# df[["rewire_id", "label_pred"]].to_csv(f"../Results/1b_lexbert_test.csv", index=False)

In [ ]:
df.to_csv(f"../Results/1b_lexbert_train.csv", index=False)